In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score 

In [ ]:
country_codes_file = "country_codes_V202001.csv" 
product_codes_path = "product_codes_HS12_V202001.csv"
baci_files = {
    2016: "BACI_HS12_Y2016_V202001.csv",
    2017: "BACI_HS12_Y2017_V202001.csv",
    2018: "BACI_HS12_Y2018_V202001.csv"
}

In [ ]:
def safe_read_csv(file_path):
    """Automatically try common encodings to read CSV; check file path validity"""
    encodings = ["GBK", "GB2312", "utf-8", "latin-1"]  # Prioritize GBK/GB2312 for Chinese-language files
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"Successfully read {file_path} (Encoding: {encoding})")
            return df
        except UnicodeDecodeError:
            continue
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {file_path}. Please check if the path is correct.")
    raise ValueError(f"Failed to read {file_path} with all tested encodings. Verify the file encoding via Notepad.")


In [ ]:
country_codes = safe_read_csv(country_codes_file)
product_codes = safe_read_csv(product_codes_path)

In [ ]:
baci_list = []
for year, file in baci_files.items():
    df = safe_read_csv(file)
    # Critical validation: Trade data must contain "k" (HS6 code) and "v" (export value) columns
    required_cols = ["i", "j", "k", "v"]  # i=exporter code, j=importer code, k=HS6 code, v=export value
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise KeyError(f"Trade file {file} is missing required columns: {missing_cols}. Check raw data column names.")
    # Add year column to mark data's year
    df["year"] = year
    baci_list.append(df)

In [ ]:
baci_df = pd.concat(baci_list, ignore_index=True)
print(f"Merged trade data size: {baci_df.shape[0]} rows × {baci_df.shape[1]} columns")

In [ ]:
baci_df["k"] = baci_df["k"].astype(str).str.zfill(6) 
baci_df["hs2"] = baci_df["k"].str[:2]  

In [ ]:
if baci_df["hs2"].isnull().all():
    raise ValueError("HS2 column generation failed. Check if the 'k' column is in valid string format.")
print(f"HS2 column generation example:\n{baci_df[['k', 'hs2']].head(3)}")

In [ ]:
hs2_export = baci_df.groupby(["i", "year", "hs2"])["v"].sum().reset_index()

In [ ]:
total_export = hs2_export.groupby(["i", "year"])["v"].sum().reset_index(name="total_v")

In [ ]:
hs2_export = pd.merge(hs2_export, total_export, on=["i", "year"])
hs2_export["export_share"] = hs2_export["v"] / hs2_export["total_v"].replace(0, np.nan)
hs2_export["export_share"] = hs2_export["export_share"].fillna(0)  、
print(f"HS2 export share calculation completed. Data size: {hs2_export.shape[0]} rows")

In [ ]:
share_matrix = hs2_export.pivot_table(
    index=["i", "year"],  
    columns="hs2",        
    values="export_share",
    fill_value=0          
)
print(f"Export share matrix dimensions: {share_matrix.shape[0]} country-years × {share_matrix.shape[1]} HS2 categories")

In [ ]:
scaler = StandardScaler()  
share_matrix_scaled = pd.DataFrame(
    scaler.fit_transform(share_matrix),
    index=share_matrix.index,
    columns=share_matrix.columns
)
# Save normalized data (optional)
share_matrix_scaled.to_csv("hs2_export_share_scaled.csv", encoding="utf-8-sig")
print(f"Data normalization completed. Results saved to hs2_export_share_scaled.csv")

In [ ]:
X = share_matrix_scaled.values
inertia = []
k_range = range(2, 11)  # Test 2-10 clusters
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)  # n_init=10 to avoid local optima
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(k_range, inertia, marker="o", linestyle="-", color="#1f77b4")
plt.xlabel("Number of Clusters (k)", fontsize=12)
plt.ylabel("Inertia (Within-cluster Sum of Squares)", fontsize=12)
plt.title("Elbow Method for Optimal k", fontsize=14)
plt.grid(alpha=0.3)
plt.savefig("elbow_plot.png", dpi=300, bbox_inches="tight")
plt.close()
print(f"Elbow plot saved to elbow_plot.png (For selecting optimal k)")

In [ ]:
optimal_k = 5  
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X)

In [ ]:
sil_score = silhouette_score(X, cluster_labels)
print(f"K-means clustering completed (k={optimal_k}). Silhouette Score: {sil_score:.3f}")

In [ ]:
cluster_result = share_matrix_scaled.reset_index()
cluster_result["cluster"] = cluster_labels

In [ ]:
if "country_code" not in country_codes.columns:
    raise KeyError("country_codes.csv is missing 'country_code' column. Check code table columns.")
cluster_result = pd.merge(
    cluster_result,
    country_codes[["country_code", "country_name_full"]],
    left_on="i",
    right_on="country_code",
    how="left"
)

In [ ]:
final_result = cluster_result[["country_name_full", "i", "year", "cluster"]].copy()
final_result["country_name_full"] = final_result["country_name_full"].fillna("Unknown Country")
final_result.to_csv("country_hs2_cluster_result.csv", index=False, encoding="utf-8-sig")
print(f"Clustering results saved to country_hs2_cluster_result.csv")
print(f"Cluster sample distribution:\n{final_result['cluster'].value_counts().sort_index()}")


hs2_export[["i", "year", "hs2", "v", "total_v", "export_share"]].to_csv(
    "hs2_export_share_raw.csv", index=False, encoding="utf-8-sig"
)